In [7]:
import pickle
import os
path = os.getcwd()
import geopandas as gpd # GeoPandas(지오판다스)
import numpy as np

In [8]:


class Rich():
    def __init__ (self, rich_did, lu_rch_did, ru_rch_did, stream_flux, c_area, geometry):
        self.rich_did = rich_did
        self.lu_rch_did = lu_rch_did
        self.ru_rch_did = ru_rch_did
        self.stream_flux = stream_flux               ## 단위 m3/s
        self.c_area = c_area
        self.geometry = geometry
        self.stream_flux_estimated = None
        self.lu_child = None
        self.ru_child = None
        self.stp_family = None
        self.stp_site = False

        self.parent = None

        self.velocity = None                         ## 단위 m/s
        self.velocity_estimated = None
        self.rch_len = None
        self.r_time = None                           ## 단위 hour

        self.PEC_est = None
        self.mass_g = None


    
    def take_lu_child(self,lu_rich_class,):
        self.lu_child = lu_rich_class
        

    def take_ru_child(self,ru_rich_class,):
        self.ru_child = ru_rich_class

    def estimate_flux(self, parents_true_flux, parents_true_carea):
        # ~np.isnan(self.stream_flux_estimated)
        if self.stream_flux is None:
            self.stream_flux_estimated = parents_true_flux *(self.c_area/parents_true_carea)

        elif self.stream_flux is not None:
            print('참값 있음')

#### hyd_monitoring_df_filter_site_3 픽스되었다는 가정 하에 농도 추정치 계산 ####
######## 현제 받은 데이터 STP 지점 인수구 있는것 143개인데 확인해보기
######## ㅇ=



line_gdf_rtime = pickle.load(open(os.path.join(path,"data","lind_gdf_rtime.p",), "rb"))
stp_gdf_info = pickle.load(open(os.path.join(path,"data","stp_gdf_pop.p",), "rb"))

k = 0.00193 # 반응속도상수 반감기 역수 h-1

koc = 484800 # 사용자 입력값, L/kg
Foc_susp = 0.1 # kg_oc*kg_solid-1
SUSP_water = 4 # mg_solid*L_water-1
constant_term = koc * Foc_susp * SUSP_water * (10**-6)   # 무차원 상수


removal_ratio = 0.641 # 물질별로 계산해서 나오는 값 제거율
base_c = 0.0000044 # 물질별로 나오는 값인것으로 알고있음, 배경농도 g/m3
e_p = 0.000085 #인당 물질 배출량,  g/(d*명)

# mass_g

def cal_mass_down(stp_r, mass_g_out, k_constant):
    if stp_r.parent is not None:
        down_mass = mass_g_out * np.exp(-1*k_constant*stp_r.r_time) #구간 잔존률 * 위에서온 mass

        if stp_r.parent.mass_g is None:
            stp_r.parent.mass_g = down_mass  
        else:
            before_mass = stp_r.parent.mass_g
            stp_r.parent.mass_g = before_mass + down_mass

        cal_mass_down(stp_r.parent, down_mass, k_constant)
    

# 우선 농도계산은 하지 않고 강줄기별 mass만 계산
for i in range(len(stp_gdf_info)):
    i_info = stp_gdf_info.loc[i,['Effluent', '인구수', 'RCH_DID']]
    
    population = i_info.인구수
    stp_r = line_gdf_rtime.loc[line_gdf_rtime.RCH_DID == i_info.RCH_DID].rich_class.values[0]
    if stp_r.r_time is not None:
    
        # flux = stp_r.stream_flux*86400 if stp_r.stream_flux_estimated is None else stp_r.stream_flux_estimated*86400 #   단위 m3/s이므로 86400곱해서 일단위( m3/day)로
        mass_g = (e_p * population)*(1-removal_ratio)  # g/day   _ 물질 배출량
        mass_g_out = mass_g*(1-removal_ratio)/(1 + constant_term)   # 처리효과 흡착??효과 고려한 물질 out 양


        # 총 물질량 합산 위에서 흘러온것 + stp에서 배출한 것
        try:
            mass_before = stp_r.mass_g
            stp_r.mass_g = mass_before + mass_g_out  # 앞단에서 모든 강줄기에 배경농도는 미리 더해놓음
        except:
            stp_r.mass_g = mass_g_out


        cal_mass_down(stp_r, mass_g_out, k)


# 추정 유량을 통해 농도 환산
for i in range(len(line_gdf_rtime)):
    if ~(np.isnan(line_gdf_rtime.loc[i,'유량']) & np.isnan(line_gdf_rtime.loc[i,'est_flux'])):
        i_r = line_gdf_rtime.loc[i,'rich_class']
        if i_r.mass_g is not None:
            try:
                pec_est = i_r.mass_g / (i_r.stream_flux_estimated * 86400)  #유량단위 현재 m3/s이므로 86400곱해서 day로 환산
            except:
                pec_est = i_r.mass_g / (i_r.stream_flux * 86400)


            line_gdf_rtime.loc[i,'rich_class'].PEC_est = pec_est
            line_gdf_rtime.loc[i,'pec'] = pec_est


gpd.GeoDataFrame(line_gdf_rtime.loc[:,['geometry_x','pec','r_time','유량','est_flux']], geometry = 'geometry_x').explore(tiles = "CartoDB positron", column = 'pec')

AttributeError: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from 'c:\\Users\\picop\\anaconda3\\envs\\test\\lib\\site-packages\\pandas\\_libs\\internals.cp37-win_amd64.pyd'>

In [9]:
line_gdf_rtime = pickle.load(open(os.path.join(path,"data","lind_gdf_rtime.p",), "rb"))
stp_gdf_info = pickle.load(open(os.path.join(path,"data","stp_gdf_pop.p",), "rb"))


AttributeError: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from 'c:\\Users\\picop\\anaconda3\\envs\\test\\lib\\site-packages\\pandas\\_libs\\internals.cp37-win_amd64.pyd'>